# Ai-Hub 데이터 추가 - 뉴스기사 기계독해 데이터



In [1]:
import os
import pandas as pd
import json
from tqdm import tqdm
from datasets import load_dataset

# 디렉토리 내의 모든 json 파일 가져오기
dir_path = './data' # 실제 디렉토리 경로로 변경
json_files = [pos_json for pos_json in os.listdir(dir_path) if pos_json.endswith('.json')]

data_rows = []
for json_file in tqdm(json_files):
    with open(os.path.join(dir_path, json_file), "r") as read_file:
        data = json.load(read_file)

        for item in data['data']:
            doc_title = item['doc_title']
            doc_class = item['doc_class']['code'] 
            doc_published = item['doc_published']
            
            data_rows.append({
                'text': doc_title,
                'target': doc_class,
                'publish_date': doc_published
            })

# 모든 결과를 하나의 DataFrame으로 결합
df = pd.DataFrame(data_rows)

# df 전처리 
df['year'] = df['publish_date'].apply(lambda x: x // 10000)
df = df[~df['target'].isin(['기타', '지역'])]
df['target'] = df['target'].replace({'IT과학': 0,
                                     '경제': 1,
                                     '사회': 2,
                                     '문화': 3,
                                     '국제': 4,
                                     '스포츠': 5,
                                     '정치': 6})
df['len'] = df['text'].apply(lambda x: len(x))
df = df[df['len'] <= 46].reset_index(drop=True)
df

100%|██████████| 7/7 [00:05<00:00,  1.30it/s]


,text,target,publish_date,year,len
0,"“시진핑엔 선물, 바이든엔 모욕”… 유럽의회서 막힌 中·EU투자협정",4,20210225,2021,37
1,김경수 분노…“내가 문 대통령께 반발? 참 희한한 일”,6,20210225,2021,30
2,오늘부터 사라진 네이버 ‘실검’…빈자리엔 날씨와 주가,0,20210225,2021,29
3,"경남도, 신혼 반값 임대주택 공급 확대",2,20210225,2021,21
4,작년 3분기 일자리 늘었다는데… 2030 일자리만 15만개 감소,1,20210225,2021,35
...,...,...,...,...,...
109318,문희상 입떼자 기다린듯 ‘이명박·박근혜 사면론’ 들이대는 통합당,6,20200525,2020,35
109319,"우즈, 미컬슨과 맞대결서 1홀 차 승리",5,20200525,2020,21
109320,권순우 “배에 임금 왕 자 생겼다” 몸상태 ‘최고’,5,20200525,2020,28
109321,40대 보험 ‘암·상해·실손’ 순…“본인·주변 경험과 노후 걱정 때문”,1,20200525,2020,39


In [2]:
df.to_csv('../full_hub_data.csv', index=False)

In [3]:
df['target'].value_counts()

target
2    39630
1    19753
3    19232
6    14058
5     6649
4     5427
0     4574
Name: count, dtype: int64